In [4]:
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
import pandas as pd
import matplotlib.pyplot as plt
import time, pickle
from keras.utils import to_categorical
import numpy as np
%matplotlib inline

In [5]:
nb_classes = 100

(train_data,train_label),(test_data,test_label_not_use)=cifar100.load_data()
train_dat=train_data.reshape((train_data.shape[0],3072)) #reshape to(50000,3072)
train_dat_df=pd.DataFrame(train_dat) #transform to dataframe
train_label_df=pd.DataFrame(train_label)
        
x_train_df=train_dat_df.sample(frac=0.9,axis=0) #use 90% in training set as x_train
y_train_df=train_label_df.iloc[list(x_train_df.index)]
        
bad_df = train_dat_df.index.isin(list(x_train_df.index))
x_test_df=train_dat_df[~bad_df] # remaining 10% as x_test
y_test_df=train_label_df.iloc[list(x_test_df.index)]
        
x_train=np.array(x_train_df).reshape((np.array(x_train_df).shape[0],32,32,3)).astype('float32')
x_test =np.array(x_test_df).reshape((np.array(x_test_df).shape[0],32,32,3)).astype('float32')
y_train=np.array(y_train_df).reshape((np.array(x_train_df).shape[0],1))
y_test=np.array(y_test_df).reshape((np.array(x_test_df).shape[0],1))

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [8]:
#Normalize
x_train /= 255
x_test /= 255

#labels to categorical
y_train = to_categorical(y_train, nb_classes)
y_test = to_categorical(y_test, nb_classes)

In [9]:
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda
from keras.models import Model
import tensorflow as tf

def make_parallel(model, gpu_count):

    def get_slice(data, idx, parts):
        shape = tf.shape(data)
        size = tf.concat([shape[:1] // parts, shape[1:]], axis=0)
        stride = tf.concat([shape[:1] // parts, shape[1:] * 0], axis=0)
        start = stride * idx
        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    # Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                # Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape, arguments={'idx': i, 'parts': gpu_count})(x)
                    inputs.append(slice_n)

                outputs = model(inputs)

                if not isinstance(outputs, list):
                    outputs = [outputs]

                # Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])
                    
    # merge outputs on CPU
    # 也可以设为GPU，如果CPU负载已经很大的话
    with tf.device('/cpu:0'):
        merged = []
        for outputs in outputs_all:
            merged.append(Concatenate(axis=0)(outputs))
        return Model(model.inputs, merged)

In [10]:
x = Input(shape=(32, 32, 3))
y = x
y = Convolution2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='valid')(y)

y = Convolution2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='valid')(y)

y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='valid')(y)

y = Flatten()(y)
y = Dense(units=128, activation='relu', kernel_initializer='he_normal')(y)
y = Dropout(0.5)(y)
y = Dense(units=nb_classes, activation='softmax', kernel_initializer='he_normal')(y)

# SGD (Stochastic Gradient Descent)
# lrate = 0.01
# decay = lrate / nb_epoch
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

model3 = Model(inputs=x, outputs=y, name='model1')

model3 = make_parallel(model3, 2)

model3.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
model1 (Model)                  (None, 100)          1682724     lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
__________

In [11]:
nb_epoch = 250
batch_size = 128
start = time.time()

#data augmentation
aug_gen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False,  # randomly flip images
)

aug_gen.fit(x_train)
gen = aug_gen.flow(x_train, y_train, batch_size=batch_size)
h = model3.fit_generator(generator=gen, steps_per_epoch=45000//batch_size, epochs=nb_epoch, validation_data=(x_test, y_test))
model3.save('CIFAR10_model_with_data_augmentation_dual_GPU.h5')



x_test_p=test_data.astype('float32')/255
# Prediction on test set
predict_test = model3.predict(x_test_p)
pred=np.argmax(predict_test,1)

df=pd.DataFrame(pred)
df.index.name='ids'
df.columns=['labels']
df.to_csv('final_result2.csv',header=True)

Epoch 1/250
  9/351 [..............................] - ETA: 6:56 - loss: 4.7871 - acc: 0.0078

KeyboardInterrupt: 